In [1]:
import pandas as pd
import xgboost
import shap
import RDG_interventional_classification
import RDG_gaussian_classification
import numpy as np
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

In [2]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_gunpoint
from sktime.datasets import load_italy_power_demand

X, y = load_italy_power_demand()


numpy_X = []
for i in range(0,X.shape[0]):
    numpy_X.append(X.loc[i].values[0])
X = np.asarray(numpy_X)

y = np.asarray(y)

X = (X - np.min(X))/(np.max(X)-np.min(X))
#y =  (y - np.min(y))/(np.max(y)-np.min(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
X

array([[0.29590033, 0.21276607, 0.17951236, ..., 0.52313396, 0.45108427,
        0.3734923 ],
       [0.246229  , 0.16995674, 0.14303712, ..., 0.54234481, 0.41223332,
        0.35390748],
       [0.6527679 , 0.5210177 , 0.45514259, ..., 0.83392445, 0.81745567,
        0.70217424],
       ...,
       [0.19880741, 0.16591476, 0.15358004, ..., 0.56473789, 0.47017158,
        0.39616317],
       [0.54643376, 0.53143661, 0.44145372, ..., 0.84637674, 0.80138529,
        0.75639384],
       [0.31847259, 0.22192337, 0.15755724, ..., 0.54911795, 0.45793257,
        0.42038565]])

In [4]:
feature_names = np.arange(24)

In [5]:
y

array(['1', '1', '2', ..., '2', '2', '2'], dtype='<U1')

In [6]:
reg = RocketClassifier(num_kernels=100)
reg.fit(np.asarray(X_train), np.asarray(y_train))
clf = reg
clf.predict_proba(X_test[3].reshape(1,24))[0][0]

0.0

In [7]:
from sktime.classification.deep_learning.cnn import CNNClassifier
cnn = CNNClassifier(n_epochs=20,batch_size=4,verbose=False)  
cnn.fit(X_train, y_train)
cnn.predict_proba(X_test[0].reshape(1,-1))[0][0]
cnn.score(X_test,y_test)

Metal device set to: Apple M1 Pro


2023-03-16 15:43:24.780320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-16 15:43:24.780447: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-16 15:43:24.911371: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-16 15:43:25.109323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


59/91 [==================>...........] - ETA: 0s

2023-03-16 15:43:42.674132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - 0s 2ms/step


0.9558011049723757

In [8]:
clf = cnn

In [9]:

cov = np.cov(X.T)
means = X.mean(axis=0)

minis = np.min(X,axis=0)
maxis = np.max(X,axis=0)

In [10]:
import timeit
import random
 



ground_truth = {}
for i in range(0,len(clf.coef_[0])):
    ground_truth[i] = np.abs(clf.coef_[0][i])
sorted_indexes = sorted(ground_truth, key=ground_truth.get,reverse=True)
sorted_indexes

In [34]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
e = shap.KernelExplainer(clf.predict_proba,X[0:10])
ks_shap_values = e.shap_values(X_test[0:1])[0]
print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 455522.415698
3/3 [==============================] - 0s 3ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

5240/5240 [==============================] - 9s 2ms/step
The time difference is : 9.363190624979325


The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

In [35]:
9.36*100

936.0

In [12]:
ks_shap_values[0][0]

0.012102750246316796

In [30]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
x = X_test[0]
#cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.5,X_train)
int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.2)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 454904.9144575
1/1 [==============================] - 0s 9ms/step
The time difference is : 2.7510367080103606


In [29]:
starttime = timeit.default_timer()
for a in range(0,1):
    ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    #ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X_train,cond_seps,cond_nonseps)
print("The time difference is :", timeit.default_timer() - starttime)

1/1 [==============================] - 0s 8ms/step
The time difference is : 1.2506114169955254


In [32]:
1.25*100 + 2.75

127.75

In [17]:
overall_ss_g_sum_indexes = []
overall_ss_i_sum_indexes = []
overall_ks_sum_indexes = []

overall_tree_sum_indexes = []


overall_ss_g_indexes = []
overall_ss_i_indexes = []
overall_ks_indexes = []

overall_tree_indexes = []

ss_g_scores = []
ss_i_scores = []
ks_scores = []
og_scores = []
#tree_scores = []

x = X_test[0]

cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.1,X_train)
int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.1)

print(int_nonseps)
print(cond_nonseps)
for a in range(0,100):
    x = X_test[a]
    ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X_train,cond_seps,cond_nonseps)
    
    ks_atts = {}
    for val in range(0,len(ks_shap_values[a])):
        ks_atts[val] = ks_shap_values[a][val]
    
        
    #tree_atts = {}
    #for val in range(0,len(tree_shap_values[a])):
        #tree_atts[val] = tree_shap_values[a][val]

    print(ks_atts)
    temp = {}
    for u in ss_i_atts.keys():
        temp[u] = np.abs(ss_i_atts[u])
    sorted_ss_i_indexes = sorted(temp, key=temp.get, reverse=True)
    
    temp = {}
    for u in ss_g_atts.keys():
        temp[u] = np.abs(ss_g_atts[u]) 
    sorted_ss_g_indexes = sorted(temp, key=temp.get, reverse=True)
    
    temp = {}
    for u in list(ks_atts.keys()):
        temp[u] = np.abs(ks_atts[u]) 
    print(temp)
    sorted_ks_indexes = sorted(temp, key=temp.get, reverse=True)
    
    
    #temp = {}
    #for u in tree_atts.keys():
       # temp[u] = np.abs(tree_atts[u]) 
    #sorted_tree_indexes = sorted(temp, key=temp.get, reverse=True)
    
    overall_ss_g_indexes.append(sorted_ss_g_indexes[0])
    overall_ss_i_indexes.append(sorted_ss_i_indexes[0])
    overall_ks_indexes.append(sorted_ks_indexes[0])
    #overall_og_indexes.append(sorted_og_indexes[0])
    #overall_tree_indexes.append(sorted_tree_indexes[0])
    
    total_sum = clf.predict_proba(x.reshape(1,-1))[0][0] -  clf.predict_proba(means.reshape(1,-1))[0][0]

    overall_ss_i_sum_indexes.append(sum(ss_i_atts.values()) - total_sum)
    overall_ss_g_sum_indexes.append(sum(ss_g_atts.values()) - total_sum)
    overall_ks_sum_indexes.append(sum(ks_atts.values()) - total_sum)
    #overall_og_sum_indexes.append(sum(og_atts.values()) - total_sum)
    
    
    ind = sorted_ss_g_indexes[0]
    new_x = x.copy()
    if len(list(ind)) > 1:
        for u in list(ind):
            new_x[u] = means[u]
    else:
        new_x[list(ind)[0]] = means[list(ind)[0]]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ss_g_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    ind = sorted_ss_i_indexes[0]
    new_x = x.copy()
    if len(list(ind)) > 1:
        for u in list(ind):
            new_x[u] = means[u]
    else:
        new_x[list(ind)[0]] = means[list(ind)[0]]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ss_i_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    ind = sorted_ks_indexes[0]
    new_x = x.copy()
    
    new_x[ind] = means[ind]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ks_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    

    
    """ind = sorted_tree_indexes[0]
    new_x = x.copy()
    
    new_x[ind] = means[ind]
    
    prediction = clf.predict(new_x.reshape(1,-1))
    
    tree_scores.append(np.abs(prediction - clf.predict(means.reshape(1,-1))))"""
            

1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
[]
[[19, 22]]
184/184 [==============================] - 0s 2ms/step
in here

1/1 [==============================] - 0s 8ms/step


184/184 [==============================] - 0s 2ms/step
{0: 0.012102750246316796, 1: 0.02540119715194139, 2: 0.006010617283948749, 3: 0.0, 4: 0.0006930852965323919, 5: 0.0, 6: 0.007363007157302426, 7: 0.039976041018744635, 8: 0.0043182754912699794, 9: 0.0, 10: 0.011002084898387499, 11: 0.013491924861149363, 12: -0.024956460200953462, 13: -0.006564829784380333, 14: 0.0014734210597721065, 15: -0.005690516070837481, 16: -0.00966594668986609, 17: -0.015442323821073245, 18: 0.2013581976478651, 19: 0.19647600017309652, 20: 0.04555800662215703, 21: 0.016902867495192903, 22: 0.053204371431774866, 23: 0.072295650415924}
{0: 0.012102750246316796, 1: 0.02540119715194139, 2: 0.006010617283948749, 3: 0.0, 4: 0.0006930852965323919, 5: 0.0, 6: 0.007363007157302426, 7: 0.039976041018744635, 8: 0.0043182754912699794, 9: 0.0, 10: 0.011002084898387499, 11: 0.013491924861149363, 12: 0.024956460200953462, 13: 0.006564829784380333, 14: 0.0014734210597721065, 15: 0.005690516070837481, 16: 0.00966594668986609,

184/184 [==============================] - 0s 2ms/step
in here

184/184 [==============================] - 0s 2ms/step
{0: 0.010526554941435184, 1: 0.01750210145184604, 2: 0.002861421677359276, 3: 0.0, 4: 0.0041013460134378356, 5: 0.006209417898016652, 6: 0.00464290329659601, 7: 0.02225256141669575, 8: 0.005617120551008579, 9: -0.019746267888502037, 10: -0.0005694984894082355, 11: 0.00968156357299054, 12: -0.014432612562104888, 13: -0.018237936796469994, 14: 0.011175758127268788, 15: 0.0008126280457843319, 16: 0.0, 17: 0.0035661091022397393, 18: 0.20516637826264011, 19: 0.19616661140511238, 20: 0.030124929554088537, 21: 0.03918735525384361, 22: 0.10264165299993935, 23: 0.023683270849043758}
{0: 0.010526554941435184, 1: 0.01750210145184604, 2: 0.002861421677359276, 3: 0.0, 4: 0.0041013460134378356, 5: 0.006209417898016652, 6: 0.00464290329659601, 7: 0.02225256141669575, 8: 0.005617120551008579, 9: 0.019746267888502037, 10: 0.0005694984894082355, 11: 0.00968156357299054, 12: 0.0144326125

184/184 [==============================] - 0s 2ms/step
in here

184/184 [==============================] - 0s 2ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: -0.015128021226341328, 8: -0.006353898405245859, 9: 0.010234589929949403, 10: -0.020287169684334198, 11: 0.0, 12: 0.006164974697456897, 13: 0.024586823243726286, 14: 0.0, 15: -0.045603091797607846, 16: -0.050237428473516046, 17: 0.0, 18: -0.03353499709517911, 19: -0.061693593655146046, 20: -0.06853329025288595, 21: -0.030922480731416825, 22: -0.025457665065680236, 23: -0.021658511160295157}
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.015128021226341328, 8: 0.006353898405245859, 9: 0.010234589929949403, 10: 0.020287169684334198, 11: 0.0, 12: 0.006164974697456897, 13: 0.024586823243726286, 14: 0.0, 15: 0.045603091797607846, 16: 0.050237428473516046, 17: 0.0, 18: 0.03353499709517911, 19: 0.061693593655146046, 20: 0.06853329025288595, 21: 0.030922480731416825, 22: 0.025457665065680236, 23: 0.0216

...

184/184 [==============================] - 0s 2ms/step
in here

184/184 [==============================] - 0s 2ms/step
{0: 0.012337796334567275, 1: 0.011572844718571756, 2: 0.0013790509309304089, 3: 0.0, 4: 0.0, 5: 0.002930204180136786, 6: -0.012379013515893017, 7: -0.016672941599134156, 8: 0.0, 9: -0.008024934838548742, 10: -0.0014493242352408564, 11: -0.004900800918277021, 12: 0.012966485353339385, 13: 0.044433093216652286, 14: 0.028754427559056134, 15: -0.11898941827488929, 16: -0.16583155916899192, 17: -0.027817593981571298, 18: 0.01790447182956928, 19: -0.10313157882454912, 20: -0.126102226656964, 21: 0.1043377705079562, 22: 0.0035204126078847153, 23: 0.023052943673677273}
{0: 0.012337796334567275, 1: 0.011572844718571756, 2: 0.0013790509309304089, 3: 0.0, 4: 0.0, 5: 0.002930204180136786, 6: 0.012379013515893017, 7: 0.016672941599134156, 8: 0.0, 9: 0.008024934838548742, 10: 0.0014493242352408564, 11: 0.004900800918277021, 12: 0.012966485353339385, 13: 0.044433093216652286, 14: 0.0

184/184 [==============================] - 0s 2ms/step
in here

184/184 [==============================] - 0s 2ms/step
{0: 0.01328115940035419, 1: 0.017987563085771163, 2: 0.004453602471305358, 3: 0.002218560185678547, 4: 0.00389805104127358, 5: 0.0, 6: 0.010098476404955457, 7: 0.05276266653708506, 8: 0.009498395797621196, 9: -0.009416560668685996, 10: 0.06016348424426217, 11: 0.021018531363049875, 12: -0.03685798967590981, 13: -0.01454732642755251, 14: 0.0040990999854246535, 15: -0.0290857687782182, 16: -0.07393320293798836, 17: -0.09964947793692137, 18: 0.23259879971021383, 19: 0.1915962471796444, 20: -0.00829996329241318, 21: 0.03479153159471825, 22: 0.14847894360711295, 23: 0.10281133311507817}
{0: 0.01328115940035419, 1: 0.017987563085771163, 2: 0.004453602471305358, 3: 0.002218560185678547, 4: 0.00389805104127358, 5: 0.0, 6: 0.010098476404955457, 7: 0.05276266653708506, 8: 0.009498395797621196, 9: 0.009416560668685996, 10: 0.06016348424426217, 11: 0.021018531363049875, 12: 0.0368

184/184 [==============================] - 0s 2ms/step
in here

184/184 [==============================] - 0s 2ms/step
{0: 0.0, 1: -0.0074295512397621446, 2: 0.005263839314449816, 3: 0.0, 4: 0.0, 5: 0.005130148354974539, 6: 0.0, 7: -0.007375642418595435, 8: -0.015741860703319357, 9: 0.08755263424933135, 10: 0.08806905105234407, 11: 0.01051489753720438, 12: 0.012432833413648583, 13: -0.009750916848140282, 14: -0.012950910194039562, 15: 0.05958039144338895, 16: 0.0757330184963976, 17: 0.011064313302741961, 18: 0.17064057692052118, 19: 0.22127175062450533, 20: 0.16418391489655115, 21: -0.07275057656584812, 22: -0.07908730527960398, 23: -0.06446860400308174}
{0: 0.0, 1: 0.0074295512397621446, 2: 0.005263839314449816, 3: 0.0, 4: 0.0, 5: 0.005130148354974539, 6: 0.0, 7: 0.007375642418595435, 8: 0.015741860703319357, 9: 0.08755263424933135, 10: 0.08806905105234407, 11: 0.01051489753720438, 12: 0.012432833413648583, 13: 0.009750916848140282, 14: 0.012950910194039562, 15: 0.05958039144338895, 1

184/184 [==============================] - 0s 2ms/step
in here

184/184 [==============================] - 0s 2ms/step
{0: 0.02465831620335779, 1: 0.018209018441009403, 2: 0.0021962294617624933, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.004248910547402381, 7: 0.011497854421214733, 8: 0.002826500949777384, 9: 0.010590302801290688, 10: 0.014516352938076795, 11: 0.0, 12: 0.032855982555415414, 13: 0.018744626312259455, 14: 0.013773616233632685, 15: -0.058463706526928685, 16: -0.09385539082688289, 17: -0.03020438890059724, 18: 0.20787401091323318, 19: 0.16188592828484016, 20: 0.0, 21: 0.06330606999830268, 22: 0.1436214670992977, 23: 0.08612027615576123}
{0: 0.02465831620335779, 1: 0.018209018441009403, 2: 0.0021962294617624933, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.004248910547402381, 7: 0.011497854421214733, 8: 0.002826500949777384, 9: 0.010590302801290688, 10: 0.014516352938076795, 11: 0.0, 12: 0.032855982555415414, 13: 0.018744626312259455, 14: 0.013773616233632685, 15: 0.058463706526928685, 16: 0.09385539082

1/1 [==============================] - 0s 8ms/step


In [18]:
int_nonseps

[]

In [19]:
cond_nonseps

[[19, 22]]

In [20]:
print('interventional')
print('sum')
print(np.mean(overall_ss_i_sum_indexes))
print(np.std(overall_ss_i_sum_indexes))

print('faithfulness')
print(np.mean(ss_i_scores))
print(np.std(ss_i_scores))

print('gaussian conditional')

print('sum')
print(np.mean(overall_ss_g_sum_indexes))
print(np.std(overall_ss_g_sum_indexes))

print('faithfulness')
print(np.mean(ss_g_scores))
print(np.std(ss_g_scores))

print('KS')

print('sum')
print(np.mean(overall_ks_sum_indexes))
print(np.std(overall_ks_sum_indexes))

print('faithfulness')
print(np.mean(ks_scores))
print(np.std(ks_scores))


print(" ")
print("ATTS:")

print("")

print("KS")

print(ks_atts)

print("I")

print(ss_i_atts)

print("G")

print(ss_g_atts)



interventional
sum
0.4154550449550152
0.6585001745912841
faithfulness
0.43104053
0.15798773
gaussian conditional
sum
-3.185101104080677
0.6463127976442383
faithfulness
0.3765369
0.17888607
KS
sum
4.4703483581542967e-10
5.370235641330762e-09
faithfulness
0.41385856
0.17593844
 
ATTS:

KS
{0: 0.006082809175103108, 1: 0.006438102441632591, 2: 0.0036998450352696737, 3: -0.002926980306455668, 4: 0.0, 5: 0.0, 6: -0.022596585459412642, 7: -0.011607472725182422, 8: 0.0032059333171650948, 9: 0.005505195219135975, 10: 0.0031786469316050668, 11: -0.01955802677675905, 12: 0.02390490938363669, 13: 0.06527687414650575, 14: 0.029769151848628238, 15: -0.12533076010873423, 16: -0.21110585185969347, 17: -0.0435053965564105, 18: 0.03449671965595336, 19: -0.16729298596093803, 20: -0.11136513758932949, 21: 0.1488428965554324, 22: 0.05931622874112061, 23: 0.015702308646180607}
I
{(0,): 0.015789896, (1,): 0.016277432, (2,): 0.0072835684, (3,): -0.00081157684, (4,): -0.0005840063, (5,): -0.002670467, (6,): -0

In [21]:
np.mean(clf.predict_proba(X))

274/274 [==============================] - 0s 2ms/step


0.5

In [ ]:
print(overall_ss_g_indexes)
print(overall_ss_i_indexes)
print(overall_ks_indexes)

In [ ]:
selected_x = x.copy()

In [ ]:
selected_x

In [ ]:
fig, ax = plt.subplots()
x = np.arange(0, 24)

ax.plot(x, selected_x, color='black')
ax.plot(x, means, color='blue')

#threshold = 0.1
#ax.axhline(threshold, color='green', lw=2, alpha=0.7)
ax.axvspan(18, 23, alpha=0.5, color='red')

ax.axvspan(0, 4, alpha=0.5, color='green')

ax.axvspan(12, 16, alpha=0.5, color='green')
ax.set_ylabel('Power Demand')
ax.set_xlabel('Hour of Day')
plt.axvline(x=18,color='black')
plt.savefig('time_series_2.pdf')

In [ ]:
clf.predict_proba(X_test[100].reshape(1,-1))

In [ ]:
clf.predict_proba(means.reshape(1,-1))